# Retrievals

In [1]:
#import libraries
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
import openai
import os

In [2]:
# Initialize embedding model with BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

C:\Users\Asif Computer\AppData\Local\Temp\ipykernel_8284\2987287500.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
c:\Users\Asif Computer\anaconda3\envs\conda\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load the PDF document using PyPDFLoader
loaders = PyPDFLoader("AI_Engineer_Roadmap.pdf")

# Extract pages from the loaded PDF
pages = loaders.load()

In [4]:
pages[9]

Document(metadata={'source': 'AI_Engineer_Roadmap.pdf', 'page': 9}, page_content=' \n   \ncodebasics.io  \n \n10 \no Track B (Affordable Fees):  \n▪ Included in  the above Master Machine Learning for Data Science & AI  \n \nWeek 23, 24: Machine Learning Projects  with Deployment       \n \n• You need to finish two end to end ML projects. One on Regression , the other on \nClassification  \n• Regression Project: Bangalore property price prediction  \no YouTube playlist link: https://bit.ly/3ivycWr  \no Project covers following  \n▪ Data cleaning  \n▪ Feature engineering  \n▪ Model building and hyper parameter tuning  \n▪ Write flask server as a web backend  \n▪ Building website for price prediction  \n▪ Deployment to AWS  \n• Classification Project: Sports celebrity image classification  \no YouTube playlist link: https://bit.ly/3ioaMSU  \no Project covers following  \n▪ Data collection and data cleaning  \n▪ Feature engineering and model training  \n▪ Flask server as a web backend  \n▪

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
# Create an empty list to store processed document chunks
doc_list = []

# Iterate over each page in the extracted pages
for page in pages:
    # Split the page content into smaller chunks
    pg_split = text_splitter.split_text(page.page_content)

    # Iterate over each chunk and create Document objects
    for pg_sub_split in pg_split:
        # Metadata for each chunk, including source and page number
        metadata = {"source": "AI policy", "page_no": page.metadata["page"] + 1}

        # Create a Document object with content and metadata
        doc_string = Document(page_content=pg_sub_split, metadata=metadata)

        # Append the Document object to the list
        doc_list.append(doc_string)

In [ ]:
qdrant_url = ""
qdrant_key = ""
collection_name = ""